In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# Reading the data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [3]:
df_train.head()

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,31103489027986,26-04-2010,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,10088999935915,21-06-2009,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,40148966010272,01-05-2013,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6
3,37157708563676,28-09-2015,20825,20007,19115,RESTAURANT,3661,id-11235901,id_1890134,81859.0,CANVASS RE-INSPECTION,31.0,Medium,locid11839,4
4,47478049564374,09-12-2015,2136,16867,10409,RESTAURANT,7876,id-11235901,id_1890134,81886.0,COMPLAINT,30.0,High,locid12264,4


The entire data available here is encoded.

In [4]:
df_train.shape

(147443, 15)

#### Checking the cities and states available in training data

In [5]:
df_train['City'].unique().tolist()

['id-11235901', 'id-11275913']

In [6]:
df_train.groupby(['City', 'State'])['State'].count()

City         State     
id-11235901  id_1890134    147177
             id_1890135        19
id-11275913  id_1890134       225
             id_1890135        22
Name: State, dtype: int64

#### Checking the cities available in test data

In [7]:
df_test['City'].unique().tolist()

['id-11235901', 'id-11275913']

In [8]:
df_test.groupby(['City', 'State'])['State'].count()

City         State     
id-11235901  id_1890134    49057
             id_1890135        4
id-11275913  id_1890134       87
Name: State, dtype: int64

The data is collected from two cities i.e. `id-11235901` and `id-11275913`.

## EDA

### 1. Handling data type

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147443 entries, 0 to 147442
Data columns (total 15 columns):
ID                    147443 non-null int64
Date                  147443 non-null object
LicenseNo             147443 non-null int64
FacilityID            147443 non-null int64
FacilityName          147443 non-null int64
Type                  143958 non-null object
Street                147443 non-null int64
City                  147443 non-null object
State                 147443 non-null object
LocationID            147408 non-null float64
Reason                147443 non-null object
SectionViolations     108375 non-null float64
RiskLevel             147443 non-null object
Geo_Loc               147443 non-null object
Inspection_Results    147443 non-null int64
dtypes: float64(2), int64(6), object(7)
memory usage: 16.9+ MB


In [6]:
temp_df = df_train.copy()
temp_df['Date'] = pd.to_datetime(temp_df['Date'], errors='coerce')

temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147443 entries, 0 to 147442
Data columns (total 15 columns):
ID                    147443 non-null int64
Date                  147390 non-null datetime64[ns]
LicenseNo             147443 non-null int64
FacilityID            147443 non-null int64
FacilityName          147443 non-null int64
Type                  143958 non-null object
Street                147443 non-null int64
City                  147443 non-null object
State                 147443 non-null object
LocationID            147408 non-null float64
Reason                147443 non-null object
SectionViolations     108375 non-null float64
RiskLevel             147443 non-null object
Geo_Loc               147443 non-null object
Inspection_Results    147443 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6), object(6)
memory usage: 16.9+ MB


In Date column some of the values are not converted to datetime

In [7]:
temp_df[temp_df['Date'].isna()]

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
2834,14542503087757,NaT,22783,5173,4932,DAYCARE (2 - 6 YEARS),15570,id-11235901,id_1890134,81871.0,CANVASS,32.0,High,locid16453,4
6570,13144807998644,NaT,245,14302,13616,RESTAURANT,6693,id-11235901,id_1890134,81867.0,COMPLAINT RE-INSPECTION,NaN,High,locid9396,4
7261,14265186052801,NaT,35745,26486,25275,RESTAURANT,17061,id-11235901,id_1890134,81865.0,COMPLAINT,2.0,Medium,locid1366,5
7815,45339938455788,NaT,3991,17567,16781,RESTAURANT,16145,id-11235901,id_1890134,81876.0,CANVASS RE-INSPECTION,10.0,High,locid16661,1
8692,19021475613488,NaT,16056,5929,5670,RESTAURANT,13005,id-11235901,id_1890134,81875.0,LICENSE RE-INSPECTION,NaN,Low,locid14611,4
13420,32321256572009,NaT,33492,6674,6384,RESTAURANT,561,id-11235901,id_1890134,81852.0,CANVASS,30.0,High,locid6337,4
14111,24523706961182,NaT,11459,7313,6961,RESTAURANT,1372,id-11235901,id_1890134,81859.0,CANVASS,NaN,High,locid11285,6
15852,16201365838172,NaT,8576,11024,10454,RESTAURANT,1173,id-11235901,id_1890134,81911.0,CANVASS RE-INSPECTION,32.0,High,locid16551,4
16863,36165257047011,NaT,33576,1484,1331,SCHOOL,1855,id-11235901,id_1890134,81852.0,CANVASS,33.0,High,locid6531,4
18142,13410066383187,NaT,2841,4490,4290,RESTAURANT,1325,id-11235901,id_1890134,81860.0,CANVASS RE-INSPECTION,31.0,High,locid3685,4


Taking these indices and checking the actual in dataframe

In [8]:
nat_indices = temp_df[temp_df['Date'].isna()].index

df_train.loc[nat_indices, 'Date'].unique()

del temp_df

In [9]:
df_train['Date'] = df_train['Date'].str.replace('29-02-2011', '28-02-2011')
df_train['Date'] = df_train['Date'].str.replace('29-02-2015', '28-02-2015')

In [10]:
df_train['Date'] = pd.to_datetime(df_train['Date'])

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147443 entries, 0 to 147442
Data columns (total 15 columns):
ID                    147443 non-null int64
Date                  147443 non-null datetime64[ns]
LicenseNo             147443 non-null int64
FacilityID            147443 non-null int64
FacilityName          147443 non-null int64
Type                  143958 non-null object
Street                147443 non-null int64
City                  147443 non-null object
State                 147443 non-null object
LocationID            147408 non-null float64
Reason                147443 non-null object
SectionViolations     108375 non-null float64
RiskLevel             147443 non-null object
Geo_Loc               147443 non-null object
Inspection_Results    147443 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6), object(6)
memory usage: 16.9+ MB


In [12]:
df_train.sort_values(by='Date')

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
37928,19858874745848,2009-01-02,12039,1767,1591,RESTAURANT,10859,id-11235901,id_1890134,81864.0,LICENSE,38.0,High,locid1995,1
124019,28147949173515,2009-01-02,33956,12013,11425,LIQUOR,7235,id-11235901,id_1890134,81902.0,CONSULTATION,33.0,Low,locid12088,5
123756,39573813914408,2009-01-02,8682,14182,13501,RESTAURANT,4833,id-11235901,id_1890134,81861.0,CANVASS RE-INSPECTION,30.0,High,locid5609,4
63098,27990829078231,2009-01-02,11939,217,202,GROCERY STORE,2598,id-11235901,id_1890134,81854.0,LICENSE RE-INSPECTION,NaN,Low,locid4081,4
51042,10182285270047,2009-01-02,5149,5983,5722,RESTAURANT,8417,id-11235901,id_1890134,81847.0,CANVASS,35.0,High,locid7350,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87958,23638000382438,2018-12-11,25743,10011,14218,RESTAURANT,4541,id-11235901,id_1890134,81892.0,CANVASS RE-INSPECTION,NaN,High,locid10746,4
50602,34539858644397,2018-12-11,33255,5104,4864,RESTAURANT,5460,id-11235901,id_1890134,81849.0,LICENSE RE-INSPECTION,NaN,Low,locid6844,4
3836,24771737801274,2018-12-11,31644,26643,25412,LIQUOR,2027,id-11235901,id_1890134,81854.0,LICENSE,3.0,Medium,locid4237,1
144139,40576090227648,2018-12-11,25717,19481,18607,RESTAURANT,2899,id-11235901,id_1890134,81892.0,CANVASS,NaN,High,locid10192,3


### 2. Checking columns to drop

In [34]:
for col in df_train.columns:
    print(col, df_train[col].nunique())

ID 147443
Date 2506
LicenseNo 34384
FacilityID 25535
FacilityName 24348
Type 409
Street 17290
City 2
State 2
LocationID 104
Reason 17
SectionViolations 61
RiskLevel 4
Geo_Loc 16316
Inspection_Results 7


In [35]:
df_train = df_train.drop(columns=['ID'], errors='ignore')

In [9]:
temp_df = df_train['Geo_Loc'].value_counts()

In [10]:
temp_df = temp_df.reset_index()

temp_df

,index,Geo_Loc
0,locid16551,1991
1,locid16811,516
2,locid3229,412
3,locid7485,356
4,locid7295,346
...,...,...
16311,locid15910,1
16312,locid6889,1
16313,locid16479,1
16314,locid10385,1


In [11]:
temp_df[temp_df['Geo_Loc'] < 10]

,index,Geo_Loc
6366,locid15031,9
6367,locid7891,9
6368,locid6072,9
6369,locid2332,9
6370,locid6272,9
...,...,...
16311,locid15910,1
16312,locid6889,1
16313,locid16479,1
16314,locid10385,1


In [12]:
temp_df['Geo_Loc'].unique()

array([1991,  516,  412,  356,  346,  343,  286,  243,  213,  203,  195,
        194,  180,  170,  150,  142,  134,  129,  116,  101,   97,   95,
         94,   93,   91,   90,   87,   80,   79,   78,   77,   75,   74,
         73,   72,   69,   67,   66,   64,   59,   57,   56,   54,   53,
         52,   51,   50,   49,   48,   47,   46,   45,   44,   43,   42,
         41,   40,   39,   38,   37,   36,   35,   34,   33,   32,   31,
         30,   29,   28,   27,   26,   25,   24,   23,   22,   21,   20,
         19,   18,   17,   16,   15,   14,   13,   12,   11,   10,    9,
          8,    7,    6,    5,    4,    3,    2,    1])

In [15]:
df_train.shape[0]

147443

In [17]:
30/df_train.shape[0]

0.00020346845899771437

In [19]:
temp_df.loc[temp_df['Geo_Loc'] < 30]

,index,Geo_Loc
230,locid163,29
231,locid13732,29
232,locid4059,29
233,locid7120,29
234,locid7344,29
...,...,...
16311,locid15910,1
16312,locid6889,1
16313,locid16479,1
16314,locid10385,1


### Inspection results by year

In [ ]:
df_train.